# Capítulo 9: Tipos de datos

## Observaciones

dtypes no es nativo de pandas, ellos son el resultado de la arquitectura cercana y emparejada con numpy.

El dtype de una columna no de cualquier forma tenemos que correlacionar a los tipos de python de el objeto contenido en la columna.

Aquí nosotros tenemos una `pd.Series` de pandas con numeros *flotantes*. Los dtypes deberan ser flotantes.

Entonces nosotros usaremos `astype` para *castear* este objeto.

In [1]:
import pandas as pd

pd.Series([1.,2.,3.,4.,5.]).astype(object)

0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
dtype: object

El dtype es ahora un objeto, pero los objetos en la lista aun son un float. Logico si usted sabe que en python, todo es un objeto y puede ser upcasteado a ser objeto.

In [2]:
type(pd.Series([1.,2.,3.,4.,5.]).astype(object)[0])

float

El dtype es ahora un objeto, pero el tipo de entradas en la lista son strings. Esto es poque numpy no hace tratos con strings y por lo tanto actúa como si fueran sólo objetos y sin importancia. 

In [3]:
type(pd.Series([1.,2.,3.,4.,5.]).astype(str)[0])

str

No confíes en los dtypes, son un artefacto de un defecto arquitectónico en los pandas. Especifíquelos como usted debe, pero no confíe en el dtype que se establece en una columna.

### Ejemplos:

#### Chequeando los tipos de columnas

Tipos de columnas pueden ser chequeados por el atributo `.dtypes` de los DataFrames.

In [4]:
df = pd.DataFrame({'A':[1,2,3],'B':[1.0,2.0,3.0],'C':[True,False,True]})
df

,A,B,C
0,1,1.0,True
1,2,2.0,False
2,3,3.0,True


In [5]:
df.dtypes

A      int64
B    float64
C       bool
dtype: object

Para una unica serie, usted puede usar el atributo `.dtype`

In [6]:
df['A'].dtype

dtype('int64')

#### Cambiando dtypes

el método `astype()` cambia el dtype de una Serie y retorna una nueva serie.

In [7]:
df = pd.DataFrame({'A': [1,2,3],'B': [1.0,2.0,3.0],'C':['1.1.2010','2.1.2011','3.1.2011'],'D':['1 Day','2 Days','3 Days'], 'E':['1','2','3']})
df


,A,B,C,D,E
0,1,1.0,1.1.2010,1 Day,1
1,2,2.0,2.1.2011,2 Days,2
2,3,3.0,3.1.2011,3 Days,3


In [8]:
df.dtypes

A      int64
B    float64
C     object
D     object
E     object
dtype: object

Cambia el tipo de columna A a float, y la columna B a entero:

In [9]:
df['A'].astype('float')

0    1.0
1    2.0
2    3.0
Name: A, dtype: float64

In [10]:
df['B'].astype('int')

0    1
1    2
2    3
Name: B, dtype: int64

El método `astype()` es para especificar el tipo a convertir (ejemplo: usted puede especificar `.astype('float64')`, `.astype('float32')` o `.astype('float16')`

Para una conversión general, usted puede usar `pd.to_numeric`, `pd.to_datetime` y `pd.to_timedelta`.

## Cambiando el tipo a numerico

`pd.to_numeric` cambia el valor a un tipo numerico.

In [11]:
pd.to_numeric(df['E'])

0    1
1    2
2    3
Name: E, dtype: int64

Por defecto, `pd.to_numeric` levanta un error si una entrada no puede ser convertido a un numero. Usted puede cambiar el comportamiento usando los parametros de *errores*.

In [12]:
pd.to_numeric(pd.Series(['1','2','a']), errors='ignore')

0    1
1    2
2    a
dtype: object

In [13]:
pd.to_numeric(pd.Series(['1','2','a']), errors='coerce')

0    1.0
1    2.0
2    NaN
dtype: float64

Si usted quiere comprobar todas las filas con una entrada que no puede ser convertida a numerico use: `indexacion booleana` con `isnull`:

In [32]:
df = pd.DataFrame({'A': [1,'x','z'],'B':[1.0,2.0,3.0],'C':[True,False,True], 'D':['1.1.2010','2.1.2011','3.1.2011'], 'E':['1 Day','2 Days','3 Days']})
df

,A,B,C,D,E
0,1,1.0,True,1.1.2010,1 Day
1,x,2.0,False,2.1.2011,2 Days
2,z,3.0,True,3.1.2011,3 Days


In [17]:
pd.to_numeric(df.A,errors='coerce').isnull()

0    False
1     True
2     True
Name: A, dtype: bool

In [18]:
df[pd.to_numeric(df.A,errors='coerce').isnull()]

,A,B,C
1,x,2.0,False
2,z,3.0,True


## Cambiando el tipo a Datetime

In [33]:
pd.to_datetime(df['D'])

0   2010-01-01
1   2011-02-01
2   2011-03-01
Name: D, dtype: datetime64[ns]

> Nota: Tenga en cuenta que 2.1.2011 se convierte al 1 de febrero de 2011. Si en su lugar desea el 2 de enero de 2011,
Es necesario utilizar el parámetro `dayfirst`.

In [21]:
pd.to_datetime('2.1.2011', dayfirst=True)

Timestamp('2011-01-02 00:00:00')

### Cambiando el tipo a timedelta

In [34]:
pd.to_timedelta(df['E'])

0   1 days
1   2 days
2   3 days
Name: E, dtype: timedelta64[ns]

### Seleccionando columnas basado en el dtype

El método `select_dtypes` puede ser usado para seleccionar columnas basado en dtype.

In [23]:
df = pd.DataFrame({'A': [1, 2, 3], 'B': [1.0, 2.0, 3.0], 'C': ['a', 'b', 'c'],
                'D': [True, False, True]}) 
df

,A,B,C,D
0,1,1.0,a,True
1,2,2.0,b,False
2,3,3.0,c,True


Con los parametros `include` y `exclude` pueden especificar cuales tipos desee:

In [24]:
df.select_dtypes(include=['number']) # Usted necesita usar una lista

,A,B
0,1,1.0
1,2,2.0
2,3,3.0


In [25]:
df.select_dtypes(include=['number', 'bool'])

,A,B,D
0,1,1.0,True
1,2,2.0,False
2,3,3.0,True


In [26]:
df.select_dtypes(include=['number', 'bool'], exclude=['int64'])

,B,D
0,1.0,True
1,2.0,False
2,3.0,True


### Resumiendo dtypes

El método `get_dtype_counts` puede ser usado para ver un desglose de tipos.

> Nota: `get_dtype_counts` a sido reemplazado por `dtypes.value_counts()`

In [29]:
df = pd.DataFrame({'A': [1, 2, 3], 'B': [1.0, 2.0, 3.0], 'C': ['a', 'b', 'c'],
            'D': [True, False, True]})

In [31]:
df.dtypes.value_counts()

int64      1
float64    1
object     1
bool       1
Name: count, dtype: int64